```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
```

```{=latex}
\title{PyHamcrest: Check What You Want to Check}
\author{Moshe Zadka -- https://cobordism.com}
\date{2020}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

In [13]:
from hamcrest import *

In [14]:
import contextlib

@contextlib.contextmanager
def show_assert():
    try:
        yield
    except AssertionError as exc:
        print(exc)

In [15]:
with show_assert():
    assert False, "hello"

hello


```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone

\end{frame}
```

## Unit tests and Assertions

### Rough definition of unit tests

```{=latex}
\begin{frame}
\frametitle{What Is a Unit Test?}

\pause

Isolation? \pause

Fast? \pause

Small System-Under-Test?


\end{frame}
```

### Why assert?

```{=latex}
\begin{frame}
\frametitle{Python Unit Test Anatomy}

\begin{itemize}
\item Runner (nose, pytest, virtue, etc.)
\item Test case (unittest.TestCase, testtools.TestCase, etc.)
\item Assertions (pytest assert rewriting, unittest.TestCase.assert_, testtools.TestCase.assert_that)
\end{itemize}

\end{frame}

```

```{=latex}
\begin{frame}
\frametitle{All-in-one or A-la-carte?}

\begin{itemize}
\item All in one: pytest, testtools, unittest
\item A-la-carte runners: nose, virtue
\item A-la-carte assertions: hamcrest
\end{itemize}

\end{frame}

```

```{=latex}
\begin{frame}
\frametitle{Hamcrest Composability}

\begin{itemize}
\item Works in any runner
\item Works with any test case
\item Focuses on just assertions
\end{itemize}

\end{frame}

```

### Bad test one: False alarm

```{=latex}
\begin{frame}
\frametitle{Bad Unit Test One: False Alarm}

AKA: False positive, Type I Error, Boy Who Cried Wolf

Asserting things that don't have to be true

\end{frame}

```

### Bad test two: Missing alarm

```{=latex}
\begin{frame}
\frametitle{Bad Unit Test Two: Missing Alarm}

AKA: False negative, Type II Error, Boy Who Cried Wolf But Nobody Believed Him

Not asserting things that have to be true

\end{frame}

```

### Thinking of unit test value

```{=latex}
\begin{frame}
\frametitle{Unit Test (Suite) Value}
```

In [16]:
def f_score(
        beta,          # 1 if False Alarms / Missing Alarms are equally bad
                       # Other common values: 2 (Missing Alarms matter more),
                       #                      0.5 (False Alarms matter more)
        true_alarm,    # Test runs that caught a bug
        false_alarm,   # Test runs that failed without a bug
        missing_alarm, # Bugs not caught
    ):
        numerator = (1 + beta**2) * true_alarm
        denominator = numerator + beta**2 * missing_alarm + false_alarm
        return numerator / denominator

```{=latex}
\end{frame}
```

## PyHamcrest Simple Examples

### Equality

```{=latex}
\begin{frame}
\frametitle{Equality}
```

In [17]:
with show_assert():
    assert_that(1, equal_to(2))


Expected: <2>
     but: was <1>



```{=latex}
\end{frame}
```

### Sequence containment

```{=latex}
\begin{frame}
\frametitle{Containment}
```

In [22]:
with show_assert():
    assert_that([1, 2, 3], has_item(5))


Expected: a sequence containing <5>
     but: was <[1, 2, 3]>



```{=latex}
\end{frame}
```

### Combination: any

```{=latex}
\begin{frame}
\frametitle{Any}
```

In [24]:
with show_assert():
    assert_that(
        1,
        any_of(
          equal_to(2),
          equal_to(0),
        )
    )


Expected: (<2> or <0>)
     but: was <1>



```{=latex}
\end{frame}
```

### Combination: all

```{=latex}
\begin{frame}
\frametitle{All}
```

In [26]:
with show_assert():
    assert_that(
        [1, 2, 3],
        all_of(
          has_item(1),
          has_item(4),
        )
    )


Expected: (a sequence containing <1> and a sequence containing <4>)
     but: a sequence containing <4> was <[1, 2, 3]>



```{=latex}
\end{frame}
```

### Composability

```{=latex}
\begin{frame}
\frametitle{Compose}
```

In [28]:
with show_assert():
    assert_that(
        [[1, 2], [3, 4]],
        has_item(
          has_item(5),
        )
    )


Expected: a sequence containing a sequence containing <5>
     but: was <[[1, 2], [3, 4]]>



```{=latex}
\end{frame}
```

## PyHamcrest Precision

### Order vs. Unorder

```{=latex}
\begin{frame}
\frametitle{Order}
```

In [31]:
with show_assert():
    assert_that(
        [1, 2, 3],
        contains_exactly(1, 2, 4)
    )


Expected: a sequence containing [<1>, <2>, <4>]
     but: item 2: was <3>



```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Any Order}
```

In [33]:
with show_assert():
    assert_that(
        [1, 2, 3],
        contains_inanyorder(4, 3, 1)
    )


Expected: a sequence over [<4>, <3>, <1>] in any order
     but: not matched: <2>



```{=latex}
\end{frame}
```

### Arbitrary boolean expressions

```{=latex}
\begin{frame}
\frametitle{Boolean Expressions}
```

In [34]:
def xor(condition1, condition2):
    return all_of(
        any_of(condition1, condition2),
        any_of(not_(condition1), not_(condition2))
    )

with show_assert():
    assert_that(
        [1,2, 3],
        xor(
            has_item(1),
            has_item(2)
        )
    )


Expected: ((a sequence containing <1> or a sequence containing <2>) and (not a sequence containing <1> or not a sequence containing <2>))
     but: (not a sequence containing <1> or not a sequence containing <2>) was <[1, 2, 3]>



```{=latex}
\end{frame}
```

### Floating point

```{=latex}
\begin{frame}
\frametitle{Floating Point Numbers}
```

In [37]:
with show_assert():
    assert_that(
        0.1 + 0.2 - 0.1 - 0.2,
        close_to(1, 0.00001)
    )


Expected: a numeric value within <1e-05> of <1>
     but: <2.7755575615628914e-17> differed by <1.0>



```{=latex}
\end{frame}
```

### String checking

```{=latex}
\begin{frame}
\frametitle{Compose}
```

In [38]:
with show_assert():
    assert_that(
        "hello beautifiul world",
        string_contains_in_order("hello", "world", "i")
    )


Expected: a string containing 'hello', 'world', 'i' in order
     but: was 'hello beautifiul world'



```{=latex}
\end{frame}
```

### Dictionary checking

```{=latex}
\begin{frame}
\frametitle{Dictionary}
```

In [41]:
with show_assert():
    assert_that(
        dict(value=1),
        has_entry("value", close_to(0.5, 0.3))
    )


Expected: a dictionary containing ['value': a numeric value within <0.3> of <0.5>]
     but: was <{'value': 1}>



```{=latex}
\end{frame}
```

## Custom Matchers

### Concept

### Simple example: prime number

### Simple example use: prime number

### Example: Sequence contains in order

### Use: Sequence contains in order on dequeu

## Subtle PyHamcrest Examples

### Check output from code

### Check property on object

### Combining assertions

### Checking datastructure equivalence

## Conclusion

```{=latex}
\end{document}
```